# Refinement of labels

# Step1. Start edit_labels
(2022/06/02) Currently, the edit_labels cannot start from Jupyter with a graph as sub-process. Please run it from VS code or command prompt with `plot_type='raster'`.<BR>

#### 0. We assume you are at...
1. Run the first labeling and training. Then you have the followings...:
    - `(dlc_root)/dlc-models`
    - `(dlc_root)/evaluation-results`
    - `(dlc_root)/labeled-data`
    - `(dlc_root)/training-datasets`
    - `(dlc_root)/videos`
    - `(dlc_root)/config.yaml`
2. Inferred the trained and new videos.
    The videos locate different folder `(video_path)` from the `(dlc_root)`.
    
#### 1. Backup the labeled data for previous training.
Please make new folder and move all files including the trained video frames, `img?????.png`, and `CollectedData_(scorer).h5`.<BR>
- Example: Make `(dlc_root)/labeled-data/(video)/3_iteration-0 folder` and move all files into it.
    
#### 2. Specify input file paths in input.csv for the edit_labels tool.
You can put multiple rows and select by the second argument in `el.read_input(input_path, 1)`.
- **training_path:** The folder path that contains `CollectedData_(scorer).h5` and `Documentation_data-(project)*.pickle` under `(dlc_root)/training-datasets(iteration-?)/UnaugmentedDataSet_(project)(date)`. Those files were used for the network training.
    - **labeled_h5**: Usually, `CollectedData_(scorer).h5`
    - **labeled_for_train_pickle**: Usually, `Documentation_data-(project)*.pickle`
    
    
- **inferred path:** folder path that contains inferred_video and h5 file for the inferred coordinates.
    - **inferred_video:** video file name
    - **inferred_h5:** h5 file for inferred coordinate. Usually `(project)(corer)(project)(data)shuffle1_200000_el.h5`

#### 3. Run the following cell for the edit_labels tool
**Attention: Jupyter cannot run edit_labels tool with graph**<BR>
Once the refinement is done, the script generates `(date)-(time)-extracted` folder that contains `extracted.h5` and `.csv` and `png` image files.
    

In [1]:
import edit_labels as el
import os


if __name__ == '__main__':

    # input data
    
    input_path = 'input.csv'
    
    if os.path.exists(input_path):
        inferred_video, inferred_h5, labeled_h5, labeled_for_train_pickle = el.read_input(
            input_path, 3)
    else:
        ############################
        # example data
        # inferred video
        inferred_video = r'input_data\rpicam-01_1806_20210722_212134.mp4'
        # inferred result h5
        inferred_h5 = r'input_data\rpicam-01_1806_20210722_212134DLC_dlcrnetms5_homecage_test01May17shuffle1_200000_el.h5'
        # labeled data for training
        labeled_h5 = r'input_data\CollectedData_DJ.h5'
        # information which frame is used for training or testing
        labeled_for_train_pickle = r'input_data\Documentation_data-homecage_test01_95shuffle1.pickle'

    # video display magnification factor
    mag_factor = 1
    # set window size and position. win_y_len_axis is only for x-axis window.
    window_geo = {'win_x_len': 1000, 'win_y_len': 100, 'win_y_len_axis': 30,
                  'win_x_origin': 0, 'win_y_origin': 0}

    el.start(inferred_video=inferred_video, inferred_h5=inferred_h5,
             labeled_h5=labeled_h5, labeled_for_train_pickle=labeled_for_train_pickle,
             mag_factor=mag_factor, window_geo=window_geo, plot_type='')

video resolution: (960, 1280, 3)
total frame number: 18000
### Inferred frames is less than the original video frames ###
### self.tots is set as len(self.mdf) : 13946  frames ###
## The ratio of Nan to the entire video frames. (total:  13946  frames)
sub1snout:     0.86
sub1leftear:     0.85
sub1rightear:     0.85
sub1tailbase:     0.97
sub2snout:     0.79
sub2leftear:     0.78
sub2rightear:     0.78
sub2tailbase:     0.98
frame 1 contains nan
frame 2 contains nan
one label is added
frame =  2  is modified: Snap of Frame 2 Taken!


# Step2. Merge labeled h5 files for re-training
## This method works when the added labels come from the original video

#### 1. Read additional labeled coordinates from the extracted h5 file  
**extracted.h5** file is in the extracted folder, like "20220602-210804-extracted".

In [5]:
import os
import pandas as pd

# read additional labeled coord from extracted h5 file
added_h5_path = '20220606-193027-extracted/extracted.h5'
df_add = pd.read_hdf(added_h5_path)

# drop likelihood
df_add = df_add.drop(['likelihood'], axis=1, level=3)

# adjust multi-index index
a = ["img{:0>5d}.png".format(x) for x in df_add.index]
idx = pd.MultiIndex.from_product([['labeled-data'],['rpicam-01_1806_20210722_212134'],a])
df_add.index = idx

# set multi-index column
_a = df_add.columns.levels[0].str.replace('DLC_dlcrnetms5_homecage_test01Jun3shuffle1_50000','wi')
df_add.columns = df_add.columns.set_levels(_a, level=0)

df_add

scorer                                                         wi           \
individuals                                                  sub1            
bodyparts                                                   snout            
coords                                                          x        y   
labeled-data rpicam-01_1806_20210722_212134 img03490.png  932.448  357.702   
                                            img03498.png  928.746  357.239   
                                            img03506.png  930.621  358.101   

scorer                                                                       \
individuals                                                                   
bodyparts                                                  leftear            
coords                                                           x        y   
labeled-data rpicam-01_1806_20210722_212134 img03490.png  1009.258  368.978   
                                            img03498.png  1009.652  368.289   
                                            img03506.png  1012.634  366.929   

scorer                                                                      \
individuals                                                                  
bodyparts                                                rightear            
coords                                                          x        y   
labeled-data rpicam-01_1806_20210722_212134 img03490.png  966.423  289.033   
                                            img03498.png  969.227  288.353   
                                            img03506.png  970.841  286.963   

scorer                                                                      \
individuals                                                                  
bodyparts                                                tailbase            
coords                                                          x        y   
labeled-data rpicam-01_1806_20210722_212134 img03490.png  992.534  260.184   
                                            img03498.png  991.735  262.387   
                                            img03506.png  990.217  261.635   

scorer                                                                      \
individuals                                               sub2               
bodyparts                                                snout     leftear   
coords                                                       x   y       x   
labeled-data rpicam-01_1806_20210722_212134 img03490.png   NaN NaN   730.0   
                                            img03498.png   NaN NaN   918.0   
                                            img03506.png   NaN NaN  1062.0   

scorer                                                                    \
individuals                                                                
bodyparts                                                       rightear   
coords                                                        y        x   
labeled-data rpicam-01_1806_20210722_212134 img03490.png  917.0    681.0   
                                            img03498.png  850.0    835.0   
                                            img03506.png  726.0      NaN   

scorer                                                                    \
individuals                                                                
bodyparts                                                       tailbase   
coords                                                        y        x   
labeled-data rpicam-01_1806_20210722_212134 img03490.png  836.0    898.0   
                                            img03498.png  908.0    880.0   
                                            img03506.png    NaN    872.0   

scorer                                                           
individuals                                                      
bodyparts                                                   

#### 2. Read backup of the labeled data for previous training
`(dlc_root)/labeled-data/(video)/?_iteration-?/CollectedData_(socrer).h5`

In [6]:
# read original labeled h5 file

labeled_data_path = '(NEED SPECIFY)'
labeled_data_path = r'W:\wataru\dlc_data\homecage_test01-wi-2022-06-03\labeled-data\rpicam-01_1806_20210722_212134\3_iteration-0'

labeled_h5_path = 'CollectedData_wi.h5'
df_org = pd.read_hdf(os.path.join(labeled_data_path, labeled_h5_path))

_a = df_org.columns.levels[0].str.replace('DJ','wi')
df_org.columns = df_add.columns.set_levels(_a, level=0)
df_org

scorer                                                         wi           \
individuals                                                  sub1            
bodyparts                                                   snout            
coords                                                          x        y   
labeled-data rpicam-01_1806_20210722_212134 img00053.png  918.000  381.000   
                                            img00266.png      NaN      NaN   
                                            img00276.png      NaN      NaN   
                                            img00305.png      NaN      NaN   
                                            img00334.png      NaN      NaN   
...                                                           ...      ...   
                                            img17492.png  281.519  944.115   
                                            img17515.png      NaN      NaN   
                                            img17842.png      NaN      NaN   
                                            img17860.png      NaN      NaN   
                                            img17968.png  455.757  777.690   

scorer                                                                      \
individuals                                                                  
bodyparts                                                 leftear            
coords                                                          x        y   
labeled-data rpicam-01_1806_20210722_212134 img00053.png  986.000  322.000   
                                            img00266.png  101.000  295.000   
                                            img00276.png   91.000  327.000   
                                            img00305.png      NaN      NaN   
                                            img00334.png  133.000  638.000   
...                                                           ...      ...   
                                            img17492.png  213.000  860.000   
                                            img17515.png  118.000  593.000   
                                            img17842.png      NaN      NaN   
                                            img17860.png      NaN      NaN   
                                            img17968.png  462.038  860.248   

scorer                                                                      \
individuals                                                                  
bodyparts                                                rightear            
coords                                                          x        y   
labeled-data rpicam-01_1806_20210722_212134 img00053.png  912.000  292.000   
                                            img00266.png  158.000  243.000   
                                            img00276.png  133.000  261.000   
                                            img00305.png      NaN      NaN   
                                            img00334.png      NaN      NaN   
...                                                           ...      ...   
                                            img17492.png      NaN      NaN   
                                            img17515.png  119.000  424.000   
                                            img17842.png      NaN      NaN   
                                            img17860.png      NaN      NaN   
                                            img17968.png  479.199  801.207   

scorer                                                                      \
individuals                                                                  
bodyparts                                                tailbase            
coords                                                          x        y   
labeled-data rpicam-01_1806_20210722_212134 img00053.png  934.187  177.223   
                                            img00266.png  168.000  459.000   
                   

#### 3. Concatenate both and drop duplication

In [7]:
# concatenate
_df = pd.concat([df_add, df_org])

# drop old duplicate labels and sort
df_merged = _df[~_df.index.duplicated(keep='first')].sort_index(level=2)
df_merged

scorer                                                         wi           \
individuals                                                  sub1            
bodyparts                                                   snout            
coords                                                          x        y   
labeled-data rpicam-01_1806_20210722_212134 img00053.png  918.000  381.000   
                                            img00266.png      NaN      NaN   
                                            img00276.png      NaN      NaN   
                                            img00305.png      NaN      NaN   
                                            img00334.png      NaN      NaN   
...                                                           ...      ...   
                                            img17492.png  281.519  944.115   
                                            img17515.png      NaN      NaN   
                                            img17842.png      NaN      NaN   
                                            img17860.png      NaN      NaN   
                                            img17968.png  455.757  777.690   

scorer                                                                      \
individuals                                                                  
bodyparts                                                 leftear            
coords                                                          x        y   
labeled-data rpicam-01_1806_20210722_212134 img00053.png  986.000  322.000   
                                            img00266.png  101.000  295.000   
                                            img00276.png   91.000  327.000   
                                            img00305.png      NaN      NaN   
                                            img00334.png  133.000  638.000   
...                                                           ...      ...   
                                            img17492.png  213.000  860.000   
                                            img17515.png  118.000  593.000   
                                            img17842.png      NaN      NaN   
                                            img17860.png      NaN      NaN   
                                            img17968.png  462.038  860.248   

scorer                                                                      \
individuals                                                                  
bodyparts                                                rightear            
coords                                                          x        y   
labeled-data rpicam-01_1806_20210722_212134 img00053.png  912.000  292.000   
                                            img00266.png  158.000  243.000   
                                            img00276.png  133.000  261.000   
                                            img00305.png      NaN      NaN   
                                            img00334.png      NaN      NaN   
...                                                           ...      ...   
                                            img17492.png      NaN      NaN   
                                            img17515.png  119.000  424.000   
                                            img17842.png      NaN      NaN   
                                            img17860.png      NaN      NaN   
                                            img17968.png  479.199  801.207   

scorer                                                                      \
individuals                                                                  
bodyparts                                                tailbase            
coords                                                          x        y   
labeled-data rpicam-01_1806_20210722_212134 img00053.png  934.187  177.223   
                                            img00266.png  168.000  459.000   
                   

#### 4. Save the merged data as CollectedData_\*.h5

In [9]:
merged_h5_path = '20220606-193027-extracted/CollectedData_wi.h5'

df_merged.to_hdf(merged_h5_path,key='df_output', mode='w')

#### 5. Copy the CollectedData_\*.h5 and png image to `(dlc_root)/labeled-data/(video)/` as well as png image files from the backup folder.

---